## Setup

In [1]:
# import packages and modules
import os
import csv
from datetime import datetime
import pyodbc
import arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
import pickle
import pandas as pd
from functools import reduce

arcpy.overwriteOutput = True

# set local variables
workspace =  "C:\\GIS\PROJECT\\ThresholdEvaluation\\ThresholdEvaluation.gdb"
sdeBase   = "C:\\GIS\\DB_CONNECT\\Vector.sde"
sdeCollect= "C:/GIS/DB_CONNECT/Collection.sde"

# sde datasets
parcels    = os.path.join(sdeBase,"sde.SDE.Parcels\sde.SDE.Parcel_Master")
soils      = os.path.join(sdeBase, "sde.SDE.Soils\sde.SDE.land_capability_NRCS2007") 
localplan  = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.LocalPlan")
towncenter = os.path.join(sdeBase, "sde.SDE.Planning\sde.SDE.TownCenter")
lcv        = os.path.join(sdeBase, "sde.SDE.Soils\sde.SDE.Land_Capability_Verification")
ipes       = os.path.join(sdeCollect, "sde_collection.SDE.Parcel\sde_collection.SDE.Parcel_LTinfo_IPES")
                          
# local impervious datasets
newImp = os.path.join(workspace, "Impervious_2018")
oldImp = os.path.join(workspace, "Impervious_2010")
changeImp = os.path.join(workspace, "Impervious_Change_2010_2019")                        
# sde impervious
impervious2010Old = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2010")
impervious2010 = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2010_Cleaned")                                                      
impervious2018 = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_2019")
imperviousChng = os.path.join(sdeBase, "sde.SDE.Impervious//sde.SDE.Impervious_Change_2010_2019")
# local impervious layers


# output tables
outTable2010Old = "ImperviousOverlayAnalysis_2010_Original.csv"
outTable2010 = "ImperviousOverlayAnalysis_2010.csv"
outTable2018 = "ImperviousOverlayAnalysis_2018.csv"
outTable2018LCVHybrid = "ImperviousOverlayAnalysis_2018_LCVHybrid.csv"
outTableChng = "ImperviousOverlayAnalysis_Change.csv"
outLocation  = "C:\\GIS\PROJECT\\ThresholdEvaluation\\"

# Create Land Capability Hybrid Layer

In [ ]:
lcvslct = arcpy.SelectLayerByAttribute_management(lcv, 'NEW_SELECTION', 
                                        "LCV_IPES IN ('1A', '1B', '1C', '2', '3', '4', '5', '6', '7')")

lcvHybrid = os.path.join(workspace, "Land_Capability_Hybrid")

In [ ]:
# delete hybrid layer if it exists
if arcpy.Exists(lcvHybrid):
    arcpy.Delete_management(lcvHybrid)

# union all lc layers together to create hybrid land capability layer
arcpy.Union_analysis([[soils,3], [lcvslct, 1], [ipes, 2]], 
                     lcvHybrid, 
                     "ALL", 
                     0.1)


In [ ]:
# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(lcvHybrid, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(lcvHybrid)
sdf.head()

In [ ]:
# iterating the columns 
for col in sdf.columns: 
    print(col) 

In [ ]:
sdf.set_index('OBJECTID')

In [ ]:
sdf.LCV_IPES.unique()

In [ ]:
# Using DataFrame.drop
df = sdf[['OBJECTID','FID_land_capability_NRCS2007', 'Land_Capab', 'GISAcre', 
          'FID_Parcel_LTinfo_IPES', 'Status', 'IPESScore','IPESScoreType',
          'RelativeErosionHazardScore','RunoffPotentialScore',
          'FID_Land_Capability_Verification', 'LCV_IPES',
#           'OWNERSHIP_TYPE', 'TRPA_LANDUSE_DESCRIPTION', 'JURISDICTION_1',
#           'PLAN_NAME', 'ZONING_ID', 'ZONING_DESCRIPTION','TOWN_CENTER', 'LOCATION_TO_TOWNCENTER',
          'GISAcre','SHAPE']]

## IPES to Land Capability Translation

In [ ]:
 
df.loc[((df.FID_Parcel_LTinfo_IPES != -1), "IPES_Translated_Score")] = df['RelativeErosionHazardScore'] + df['RunoffPotentialScore']

In [ ]:
df

In [ ]:
df.rename(columns={'Land_Capab':'LandCapability_NRCS',
                   'LCV_IPES':'LandCapability_Verified',
                    'JURISDICTION_1':'JURISDICTION'}, 
                 inplace=True)

In [ ]:
df.loc[(df.IPES_Translated_Score == 0), 'Land_Capability_Modified'] = '1B'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1A'), 'Land_Capability_Modified'] = '1A'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1C'), 'Land_Capability_Modified'] = '1C'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '2'), 'Land_Capability_Modified'] = '2'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '1B'), 'Land_Capability_Modified'] = '1B'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '3'), 'Land_Capability_Modified'] = '3'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '4'), 'Land_Capability_Modified'] = '4'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '5'), 'Land_Capability_Modified'] = '5'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '6'), 'Land_Capability_Modified'] = '6'
df.loc[(df.IPES_Translated_Score > 0)   & (df.IPES_Translated_Score < 270) & (df.LandCapability_NRCS == '7'), 'Land_Capability_Modified'] = '7'
df.loc[(df.IPES_Translated_Score > 269) & (df.IPES_Translated_Score < 380), 'Land_Capability_Modified'] = '3'
df.loc[(df.IPES_Translated_Score > 379) & (df.IPES_Translated_Score < 415), 'Land_Capability_Modified'] = '4'
df.loc[(df.IPES_Translated_Score > 414) & (df.IPES_Translated_Score < 511), 'Land_Capability_Modified'] = '5'
df.loc[(df.IPES_Translated_Score > 510) & (df.IPES_Translated_Score < 600), 'Land_Capability_Modified'] = '6'
df.loc[(df.IPES_Translated_Score > 599), 'Land_Capability_Modified'] = '7'

df.loc[(df.FID_Parcel_LTinfo_IPES == -1) & (df.FID_Land_Capability_Verification.notna()), 'Land_Capability_Modified'] = df.LandCapability_Verified

df.loc[(df.FID_Parcel_LTinfo_IPES == -1) & (df.FID_Land_Capability_Verification == -1), 'Land_Capability_Modified'] = df.LandCapability_NRCS

df.loc[(df.Land_Capability_Modified.isnull()), 'Land_Capability_Modified'] = df.LandCapability_NRCS

In [ ]:
df.Land_Capability_Modified.unique()


In [ ]:
df['Land_Capability_Modified'].fillna('NA', inplace=True)

In [ ]:
df.set_index('OBJECTID')

In [ ]:
# export cleaned up lcv layer with modified field for land capability
df.spatial.to_featureclass(lcvHybrid)

In [ ]:
df.GISAcre.sum()

In [ ]:
np.round(pd.pivot_table(df, index="Land_Capability_Modified",values=["GISAcre"],aggfunc=np.sum),0)

# Land Capability/Impervious Overlay Analysis

### New 2010 Impervious Analysis

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2010")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: id_NRCS_Impervious_LocalPlan_TownCenter_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2010, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2010)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2010))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2010)

# success?
print("Table {0} created".format(outTable2010))

### 2018 Impervious Analysis

In [ ]:
lcvHybrid

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2018")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(lcvHybrid, impervious2018, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2018)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2018))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2018)

# success?
print("Table {0} created".format(outTable2018LCVHybrid))

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2018")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2018, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2018)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2018))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2018)

# success?
print("Table {0} created".format(outTable2018))

## Overlay Overlay

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_Change")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(lcvHybrid, imperviousChng, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTableChng)):
    arcpy.Delete_management(os.path.join(outLocation,outTableChng))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTableChng)

# success?
print("Table {0} created".format(outTableChng))

In [ ]:
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTableChng)):
    arcpy.Delete_management(os.path.join(outLocation,outTableChng))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTableChng)

# success?
print("Table {0} created".format(outTableChng))

## Old 2010 Overlay

In [ ]:
# output feature classes
id_NRCS_Impervious = "memory\id_NRCS_Impervious"

id_NRCS_Impervious_LocalPlan = "memory\id_NRCS_Impervious_LocalPlan"

id_NRCS_Impervious_LocalPlan_TownCenter = "memory\id_NRCS_Impervious_LocalPlan_TownCenter"

id_NRCS_Impervious_LocalPlan_TownCenter_Parcel = os.path.join(workspace,"id_NRCS_Impervious_LocalPlan_TownCenter_Parcel_2010Old")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious"):
    print("Deleting: memory\id_NRCS_Impervious")
    arcpy.Delete_management("memory\id_NRCS_Impervious")

# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan")
    
# delete in memory feature class
if arcpy.Exists("memory\id_NRCS_Impervious_LocalPlan_TownCenter"):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_TownCenter")
    arcpy.Delete_management("memory\id_NRCS_Impervious_LocalPlan_TownCenter")

# delete staging feature class
if arcpy.Exists(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel):
    print("Deleting: memory\id_NRCS_Impervious_LocalPlan_Parcel")
    arcpy.Delete_management(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)
    
# identify land capability/impervious
print("Identifying Soils with Impervious Surfaces...")
arcpy.Identity_analysis(soils, impervious2010Old, id_NRCS_Impervious)

# identify land capability/impervious/local plan
print("Identifying Soils with Impervious Surfaces & Local Plan layers...")
arcpy.Identity_analysis(id_NRCS_Impervious, localplan, id_NRCS_Impervious_LocalPlan)

# identify land capability/impervious/local plan/town center
print("Identifying Soils with Impervious Surfaces, Local Plan, & Town Center layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan, towncenter, id_NRCS_Impervious_LocalPlan_TownCenter)

# identify land capability/impervious/local plan/town center/parcels
print("Identifying Soils with Impervious Surfaces, Local Plan, Town Center, and Parcel layers...")
arcpy.Identity_analysis(id_NRCS_Impervious_LocalPlan_TownCenter, parcels, id_NRCS_Impervious_LocalPlan_TownCenter_Parcel)

# calculate acres
print("Calculating Acres...")
with arcpy.da.UpdateCursor(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, ['GISAcre', 'SHAPE@']) as cursor:
    for row in cursor:
        row[0] = row[1].getArea('GEODESIC', 'ACRES')
        cursor.updateRow(row)

# delete table if it already exists
print("Deleting Old Table...")
if arcpy.Exists(os.path.join(outLocation,outTable2010Old)):
    arcpy.Delete_management(os.path.join(outLocation,outTable2010Old))

# Execute Table To Table
print("Exporting New Table...")
arcpy.TableToTable_conversion(id_NRCS_Impervious_LocalPlan_TownCenter_Parcel, outLocation, outTable2010Old)

# success?
print("Table {0} created".format(outTable2010Old))

# Change Analysis

In [2]:
workspace   = r"C:\GIS\PROJECT\ImperviousSurface\ImperviousSurface.gdb"
sdfImp2010  =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2010_Cleaned"))
sdfImp2018  =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2018_2019"))
sdfImpChange =  pd.DataFrame.spatial.from_featureclass(os.path.join(workspace, "Impervious_2018_2019_ChangeAnalysis"))

OSError: "C:\GIS\PROJECT\ImperviousSurface\ImperviousSurface.gdb\Impervious_2010_Cleaned" does not exist

In [ ]:
sdfImpChange.info()

In [ ]:
sdfImpChange.General_Status.unique()

In [ ]:
sdfImp2018

In [ ]:
sdf